## User Management (Keycloak)
Interaction with EOEPCA endpoint using python Demo Client.

In [ ]:
import utils.DemoKeycloakClient as Client
import jwt
import json
import requests
from keycloak import KeycloakPostError

base_domain = "develop.eoepca.org"
platform_domain = f"keycloak.{base_domain}"
server_url = f"https://{platform_domain}"
realm = "master"

## Client
We instantiate an object to interact with the platform. The object dynamically registers two Keycloak clients, one with admin privileges, another to act as a resource server to demo resources protection using UMA (User Managed Access).

In [ ]:
keycloak = Client.DemoKeycloakClient(server_url, realm, "admin", "admin_Abcd1234#")

## User Authentication
User authenticates and the client receives an ID Token (JWT) that represents the user, and is used to identify the user in UMA authorization flows.

#### Create Users
Create two users, an Eric and Alice both with user role.

In [ ]:
keycloak.create_realm_role('user')
eric_id = keycloak.create_user("eric", "eric", ["user"])
alice_id = keycloak.create_user("alice", "alice", ["user"])

#### Inspect Eric User Token

In [ ]:
token = keycloak.get_user_token("eric", "eric")
print("Eric token:\n" + json.dumps(token, indent = 2))
eric_access_token = token['access_token']
jwt_header = jwt.get_unverified_header(eric_access_token)
print("JWT Header:\n" + json.dumps(jwt_header, indent = 2))
jwt_payload = jwt.decode(eric_access_token, options={"verify_signature": False})
print("JWT Payload:\n" + json.dumps(jwt_payload, indent = 2))

#### Inspect Alice User Token

In [ ]:
token = keycloak.get_user_token("alice", "alice")
print("Eric token:\n" + json.dumps(token, indent = 2))
alice_access_token = token['access_token']
jwt_header = jwt.get_unverified_header(alice_access_token)
print("JWT Header:\n" + json.dumps(jwt_header, indent = 2))
jwt_payload = jwt.decode(alice_access_token, options={"verify_signature": False})
print("JWT Payload:\n" + json.dumps(jwt_payload, indent = 2))

The ID Token (JWT) identifies the user via user_name / sub (Subject) fields, and the client via the aud (Audience) field. The JWT is signed and can be verified, using the kid (Key ID) field, via the JWKS endpoint of the Authorization Server.

## Protect resources
Access a protected resources using UMA flow

#### Assign premium role to Eric
Roles are used to define policies that will protect resources based on roles - Role-based access control (RBAC)

In [ ]:
keycloak.create_realm_role('user-premium')
keycloak.assign_realm_role_to_user(eric_id, 'user-premium')

#### Register resources
Register protected resources

In [ ]:
resources = [
    {
        "name": "Default Resource",
        "uri": "/*"
    },
    {
        'name': 'Protected Resource',
        'icon_uri': '/protected/*',
    },
    {
        "name": "Premium Resource",
        "uri": "/protected/premium/*"
    }
]
keycloak.register_resources(resources)

Right now, resources are protected by a default policy, which grants access to users within the realm.
Let's see how Keycloak protects resources using role based and user based policies.

#### Register policies
Register role based and user based policies

In [ ]:
keycloak.register_role_policy("Only Premium User Policy", "user-premium")
keycloak.register_user_policy("Only Eric User Policy", [eric_id])

#### Register resource permissions
Resources permissions are set by assigning policies to resources.

In [ ]:
permissions = [
    {
        "name": "Default Resource Permission",
        "type": "resource",
        "logic": "POSITIVE",
        "decisionStrategy": "UNANIMOUS",
        "resources": [
            "Default Resource"
        ],
        "policies": [
            "Default Policy"
        ]
    },
    {
        "name": "Premium Resource Permission",
        "type": "resource",
        "logic": "POSITIVE",
        "decisionStrategy": "UNANIMOUS",
        "resources": [
            "Premium Resource"
        ],
        "policies": [
            "Only Premium User Policy"
        ]
    },
    {
        "name": "Protected Resource Permission",
        "type": "resource",
        "logic": "POSITIVE",
        "decisionStrategy": "UNANIMOUS",
        "resources": [
            "Protected Resource"
        ],
        "policies": [
            "Only User Policy"
        ]

    }
]
keycloak.assign_resources_permissions(permissions)

#### Get PAT (Protection API token)
PAT (Protection API token) is used to access Keycloak's Protection API, which manages resources and policies.

In [ ]:
pat = keycloak.generate_protection_pat()
print(json.dumps(pat, indent=2))
access_token = pat['access_token']

### Get Resource Id
Get Premium Resource id

In [ ]:
# it's possible to query resources by many fields, including name and uri
protected_resource_id = keycloak.get_resource_id(name="Premium Resource")[0]
print(protected_resource_id)
protected_resource_id = keycloak.get_resource_id(uri="/protected/premium/*")[0]
print(protected_resource_id)

#### Get UMA access token for premium resource for both Eric and Alice

In [ ]:
uma_ticket = keycloak.create_permission_ticket(resources=[protected_resource_id])['ticket']
print('UMA ticket for resource ' + protected_resource_id + ':\n' + uma_ticket)
eric_access_token = keycloak.get_user_token("eric", "eric")['access_token']
eric_rpt = keycloak.get_rpt(eric_access_token, uma_ticket)['access_token']
print('Eric RPT:\n' + str(eric_rpt))

In [ ]:
uma_ticket = keycloak.create_permission_ticket(resources=[protected_resource_id])['ticket']
print('UMA ticket for resource ' + protected_resource_id + ':\n' + uma_ticket)
alice_access_token = keycloak.get_user_token("alice", "alice")['access_token']
try:
    alice_uma_access_token = keycloak.get_rpt(alice_access_token, uma_ticket)['access_token']
except KeycloakPostError as e:
    print(str(e))

Trying to get a UMA token for Alice results in a 403 Forbidden Error. The reason being Alice is not allowed to access the `/protected/premium/*` resource because it's protected by a `user-premium` role which Alice does not have.

#### Use Eric UMA access token to access the premium resource

In [ ]:
headers = {
    "cache-control": "no-cache",
    "Authorization": "Bearer " + eric_rpt
}
premium_resource_url = "http://localhost:8080/protected/premium"
print('GET ' + premium_resource_url)
response = requests.get(premium_resource_url, headers=headers, verify=False)
print(str(response.status_code))

The expected result is 200 (or 404 if the demo app isn't running), meaning a successfuly access to the protected resource by Eric.